# Super computer Assisted edge detection and STEM EELS

In [ ]:
%%time
# %matplotlib widget
%matplotlib ipympl

import matplotlib.pylab as plt
import numpy as np

import Pyro5.api
import io
import time


In [ ]:
# !pip install pyTEMlib

## set server

In [ ]:
%%time
uri = "PYRO:array.server@10.46.217.242:9093"
array_server = Pyro5.api.Proxy(uri)

## acquire camera

In [ ]:
%%time
#Activate camera
array_server.activate_camera()
array_list, shape, dtype = array_server.get_ds(128, 128)
im_array = np.array(array_list, dtype=dtype).reshape(shape)
# np.savez("survey_image_gold_ag.npz", img = im_array)
plt.figure()
im = plt.imshow(im_array, cmap="gray")
# plt.colorbar(im)
# plt.show()

## more - dark current correction
    - As the DM does that automatically in the backend
    - But when we acquire it we have to ourself correct for it
            - subtract the signal when the beam is blanked

## take eels

In [ ]:
%%time
## dummy eels
dispersion = .15 # eV/pixel -- 350
offset = 0 -32
exposure = 0.0005

array_server.acquire_camera(exposure = exposure)# exposure is 0.5 seconds 
array_list, shape, dtype = array_server.get_eels()
array = np.array(array_list, dtype=dtype).reshape(shape)

# min max norm
array -= array.min()
# array /= array.max()
# array = corrected_spectrum(dark_cl, gain, array)
# np.savez("eels_.npz", spec = array)

# Plot spectrum
plt.figure()
plt.plot(dispersion * np.arange(len(array))+offset, array)
plt.xlabel("Energy Loss (eV)")
plt.ylabel("Intensity (a.u.)")
plt.title("Dummy EELS Spectrum")
plt.grid(True)
# plt.ylim(0, 1e6)  # Uncomment if needed
plt.show()

## place beam and take eels

In [ ]:
x, y =  44, 101

In [ ]:
array_server.set_beam_pos(x, y)
array_server.acquire_camera(exposure=exposure)
array_list, shape, dtype = array_server.get_eels()
array = np.array(array_list, dtype=dtype).reshape(shape)
# Plot spectrum
plt.figure()
plt.plot(dispersion * np.arange(len(array))+offset, array)
plt.xlabel("Energy Loss (eV)")
plt.ylabel("Intensity (a.u.)")
plt.title("Dummy EELS Spectrum")
plt.grid(True)
# plt.ylim(0, 1e6)  # Uncomment if needed
plt.show()

## detect edges : do eels on those coordinates

In [ ]:
%%time
import numpy as np
import matplotlib.pyplot as plt

def detect_bright_region(image):
    # Calculate the gradient in the X and Y directions
    gx = np.gradient(image, axis=1)  # Gradient in X direction
    gy = np.gradient(image, axis=0)  # Gradient in Y direction
    g = np.sqrt(gx**2 + gy**2)
    g_normalized = (g / g.max()) * 255
    return g_normalized.astype(np.uint8)


edges_detected = detect_bright_region(im_array)

# Displaying the edges detected
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.imshow(im_array, cmap='gray')
plt.title('Original Image')

plt.subplot(1, 2, 2)
plt.imshow(edges_detected)
plt.title('Detected places to do eels')

plt.show()


In [ ]:
%%time
# threshold to take eels
threhold_eels = edges_detected.mean()*4
spots = edges_detected.copy()
spots[spots<threhold_eels]=0
spots[spots>0]= 1.0
print(spots.sum())
plt.figure()
plt.imshow(spots)

In [ ]:
%%time
tstart = time.time()
image_size = 128# bad idea to hardcode this
array_server.activate_camera()
scale = int(2**14/image_size)
line_p = np.zeros([image_size, image_size, array.shape[0]])

accepted = 0
for i, y in enumerate(range(image_size)):
    print("line scan ", y, )
    for j, x in enumerate(range(image_size)):
        if edges_detected[i,j]> threhold_eels: # condition to do eels
            accepted+=1
            array_server.set_beam_pos(x, y)
            array_server.acquire_camera(exposure=exposure)
            array_list, shape, dtype = array_server.get_eels()
            array = np.array(array_list, dtype=dtype).reshape(shape)
            np.savez(f"eels_nanowire{x}_{y}.npz", spec = array)
            plt.plot(array)
            line_p[i,j] = array  # summing eels to get brigh field pixel value
tend = time.time()

In [ ]:
print("accepted_points",accepted)
print("time_takenin seconds", tend - tstart)